In [2]:
import os
from bs4 import BeautifulSoup
import csv
import requests
import pandas
import spacy


In [3]:
food_data=open("food.html","r",encoding="utf8")
food=food_data.read()
len(food)

3474604

In [4]:
soup=BeautifulSoup(food,"html.parser")


In [5]:
recipes=soup.find_all('div',{'class':"fd-tile fd-recipe"})
recipes=recipes[:880]
len(recipes)

880

In [6]:
recipe_list=[]
for recipe in recipes:
    title=recipe.find('h2',{'class':"title"}).text
    links=recipe.find("a",{'href':True})['href']
    recipe_list.append({'name':title,'link':links})
len(recipe_list)    

880

In [7]:
with open("recipeLinks.csv",'w',newline='') as csvfile:
    fieldnames=['name','link']
    writer=csv.DictWriter(csvfile,fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(recipe_list)
    
    

In [12]:
recipe_list[0]['name']

'Chicken Tikka Masala'

In [22]:
import os
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import pandas as pd
import io
import requests
from PIL import Image

def download_image(download_path,url,filename):
    try:
        image_content = requests.get(url).content
        image_bytes = io.BytesIO(image_content)
        file_path = download_path + filename
        image = Image.open(image_bytes)
        with open(file_path,"wb") as f:
            image.save(f,"JPEG")
    except Exception as e:
        print("Exception is ", e)
        
        
def download_images(download_folder_path,urls,recipe,recipe_index,train_imgs_count):
    try:
        recipe_folder_name = str(recipe_index) + "_" + recipe
        train_folder_path = os.path.join(download_folder_path,recipe_folder_name,"train")
        test_folder_path = os.path.join(download_folder_path,recipe_folder_name,"test")

        if not os.path.exists(train_folder_path):
            os.makedirs(train_folder_path)
        if not os.path.exists(test_folder_path):
            os.makedirs(test_folder_path)
            
        img_number = 1
        for url in urls:
            if img_number <= train_imgs_count:
                download_image(train_folder_path,url,str(img_number)+"_"+recipe+".jpg")
            else:
                download_image(test_folder_path,url,str(img_number)+"_"+recipe+".jpg")
            img_number+=1
    except Exception as e:
        print("Exception is ", e)
        
# start web browser browser = webdriver.Chrome()

PATH="C:\\Users\\Aarush Raj\\OneDrive\\Desktop\\img2rec\\venv\\chromedriver.exe"
chrome_service=webdriver.ChromeService(executable_path=PATH)
chrome_options=Options()
chrome_options.add_experimental_option("detach",True)



# search for recipe on Google Images
"""search_url = f"https://www.google.com/search?site=&tbm=isch&source=hp&biw=1873&bih=990&q={recipe_name}"
driver.get(search_url)"""

try:
    driver=webdriver.Chrome(service=chrome_service,options=chrome_options)
    driver.maximize_window()
    driver.get('https://images.google.com/')
    time.sleep(10)
    search_box = driver.find_element(By.NAME,"q")
    # wait for search results to load
    start = 1 
    count= 1
    big_delay = 7
    small_delay = 3
    max_imgs = 10
    extra_imgs = 5
    batch_size = 10
    no_of_img_to_train=8


    df = pd.read_csv("recipeLinks.csv",skiprows= start,nrows=count,names=['name','link'])
    # download images
    for i in range(batch_size):
        recipe=df.loc[i+start-1,'name']
        recipe_index_csv=i+start-1
        
        search_box.send_keys(recipe)
        search_box.send_keys(Keys.ENTER)
        time.sleep(10)
    
        image_urls=set()
        # click on the first image
        thumbnails = driver.find_elements(By.CLASS_NAME,"mNsIhb")
        while len(thumbnails) < max_imgs+extra_imgs: # checking condition before scrolling to minimize HTTP Requests
            print("Less images obtained")
            time.sleep(small_delay)
            driver.execute_script("window.scrollTo(0,document.body.scrollHeight);")
            time.sleep(small_delay)
            thumbnails = driver.find_elements(By.CLASS_NAME,"mNsIhb") 
            
        x=len(image_urls)
        count=0
        for thumbnail in thumbnails[x:batch_size]:
            if(count==max_imgs):
                break
            
            try:
                thumbnail.click()
                # wait for image to load
                time.sleep(5)
            except:
                continue
            print(thumbnail)
            pop_up_window = driver.find_elements(By.CLASS_NAME,"jlTjKd")  
            for pop_up_elements in pop_up_window:
                img_tags=pop_up_elements.find_elements(By.TAG_NAME,"img")
                class_name=pop_up_elements.get_attribute("class")
                if "iPVvYb" in class_name.strip():
                    for img_tag in img_tags:
                        image_url=img_tag.get_attribute('src')
                        image_urls.add(image_url)
                    
            """images= driver.find_elements(By.CLASS_NAME,"iPVvYb")
            for image in images:
                if image.get_attribute('src') and 'http' in image.get_attribute('src'):
                    image_urls.add(image.get_attribute('src'))
                    print("Found Image")
            print(images) 
            driver.back()
            time.sleep(6) 
            
        for image_url in image_urls:
            response = requests.get(image_url)
            image_filename = os.path.join(download_dir, f"image_{i}.jpg")
            with open(image_filename, "wb") as f:
                f.write(response.content)
        
            print("img_elem",image_elem)
                print("\n")
                print("image thumbnail",image_th)
                print(" ")
                print("imgurl",image_url)
            """    
        # download the image
        download_images(".\\images",image_urls,recipe,recipe_index_csv,no_of_img_to_train)
                    
                
                # go back to search results
        time.sleep(7)        
    # close web browser
    #print(image_urls)
    driver.quit()
except Exception as e:
    print("Exception-",e)
    driver.quit()    

Less images obtained
Less images obtained
Less images obtained
Less images obtained
Less images obtained
Less images obtained
Less images obtained
Less images obtained
Less images obtained
Less images obtained
Less images obtained
Less images obtained
Less images obtained
Exception- Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=124.0.6367.61)
Stacktrace:
	GetHandleVerifier [0x00007FF7A3167032+63090]
	(No symbol) [0x00007FF7A30D2C82]
	(No symbol) [0x00007FF7A2F6EC65]
	(No symbol) [0x00007FF7A2F4CA7C]
	(No symbol) [0x00007FF7A2FDD687]
	(No symbol) [0x00007FF7A2FF2AC1]
	(No symbol) [0x00007FF7A2FD6D83]
	(No symbol) [0x00007FF7A2FA83A8]
	(No symbol) [0x00007FF7A2FA9441]
	GetHandleVerifier [0x00007FF7A35625AD+4238317]
	GetHandleVerifier [0x00007FF7A359F70D+4488525]
	GetHandleVerifier [0x00007FF7A35979EF+4456495]
	GetHandleVerifier [0x00007FF7A3240576+953270]
	(No symbol) [0x00007FF7A30DE54F]
	(No symbol) [0x00007FF7A30D9

In [1]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import requests
import io
from PIL import Image
import os
PATH="C:\\Users\\Aarush Raj\\OneDrive\\Desktop\\img2rec\\venv\\chromedriver.exe"
chrome_service=webdriver.ChromeService(executable_path=PATH)
driver=webdriver.Chrome(service=chrome_service)
"""driver.get("https://www.browserstack.com/guide/python-selenium-to-run-web-automation-test")
print(driver.title)
chrome_options=webdriver.ChromeService(executable_path=PATH)
chrome_options.add_argument('--start-maximized')
wd=webdriver.Chrome(options=chrome_options)
#wd=webdriver.Chrome(PATH)"""
try:
    image_url="https://cdn.britannica.com/79/232779-050-6B0411D7/German-Shepherd-dog-Alsatian.jpg"
    image_content=requests.get(image_url).content
    image_file=io.BytesIO(image_content)
    image=Image.open(image_file)
    file_path=""+"t.jpg"
    with open(file_path,"wb") as f:
        image.save(f,"JPEG")
except Exception as e:
    print("Failed with exception",e)        
    

In [40]:
driver.close()